Dealing with scoring. Will convert to scripts later 


- must check that size of y and size of scoring file the same
- 

# This jupyter file has code for all graphs and tables at the member level

In [47]:
import pandas as pd 
import numpy as np
import os 
import sys 
job = '4081278'
from code.data_processing import get_data
y = None

In [30]:
from code.data_processing import get_all_datasets
all_datasets = get_all_datasets()
datasets = {}
for d in all_datasets:
    datasets[d] = get_data(d)

First some kind of measure of how dominant a single classifier is 

# Functions to deal with dataset

In [2]:
# load a single seed of a task with ypred vs observation 
def load_scoring(fn : str) -> np.array: 
    assert('.csv' in fn)
    x = pd.read_csv(fn).transpose().to_numpy()
    assert(len(x.shape) == 2)
    return x 

In [3]:
# load each scoring dataframe for each seed
def load_all_scoring(path : str) -> np.array: 
    return np.array([load_scoring(path+'/'+x) for x in os.listdir(path) if '.csv' in x])

In [69]:
# get accuracy of a single member vector on y
def acc_member(X : np.array) -> np.array: # gets the same X for each m ember 
    assert(len(X.shape) == 1)
    assert(len(y) == len(X))
    correct = 0
    for i in range(len(y)):
        if y[i] == X[i]: # fail here 
            correct += 1
    return correct / len(X)

In [5]:
# get accuracy of a single seed
def acc_seed(X : np.array) -> np.array:
    assert(len(X.shape) == 2)
    r = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        r[i] = acc_member(X[i])
    return r

In [6]:
# get accuracy over all seeds
def acc_task(X : np.array) -> list: # list not dict for now 
    assert(len(X.shape) == 3)
    r = []
    for i in range(X.shape[0]):
        r.append(acc_seed(X[i]))
    return r

# Graphing functions 

# Now loading in 

In [7]:
scoring_task_1 = load_all_scoring('results_file/4080640/1/scoring/')
print(f'Shape of all scores in task  = {scoring_task_1.shape}, (# seeds, # members in ensemble, # observatins)')
acc_task_1 = np.array(acc_task(scoring_task_1))
print(f'Shape for each seed = {acc_task_1[0].shape}')
print(f'Shape for acc over seeds = {acc_task_1.shape}') # might fail on ragged array

Shape of all scores in task  = (30, 5, 207), (# seeds, # members in ensemble, # observatins)
Shape for each seed = (5,)
Shape for acc over seeds = (30, 5)


In [8]:
acc_task_1

array([[0.53140097, 0.51690821, 0.51690821, 0.52173913, 0.5410628 ],
       [0.52173913, 0.50724638, 0.47342995, 0.46859903, 0.47342995],
       [0.44927536, 0.49275362, 0.54589372, 0.51207729, 0.50724638],
       [0.52173913, 0.56038647, 0.50724638, 0.47342995, 0.46859903],
       [0.50241546, 0.51690821, 0.52173913, 0.55555556, 0.46376812],
       [0.47342995, 0.50241546, 0.5410628 , 0.50724638, 0.46859903],
       [0.5410628 , 0.52173913, 0.53623188, 0.50724638, 0.57487923],
       [0.50724638, 0.51690821, 0.4589372 , 0.53140097, 0.50241546],
       [0.46376812, 0.46859903, 0.51690821, 0.48309179, 0.48309179],
       [0.51690821, 0.49758454, 0.49758454, 0.48309179, 0.48309179],
       [0.57004831, 0.50724638, 0.53623188, 0.55555556, 0.57487923],
       [0.39613527, 0.45410628, 0.44444444, 0.40096618, 0.45410628],
       [0.48792271, 0.47826087, 0.51690821, 0.49758454, 0.44444444],
       [0.51207729, 0.50724638, 0.51690821, 0.55555556, 0.55072464],
       [0.5410628 , 0.5410628 , 0.

In [9]:
#df_ensemble = pd.read_csv(f'results_file/{job}/1/bagboost_experiment_{job}_task_1_cleveland.csv').describe()
#df_ensemble

In [11]:
#mean_acc = df_ensemble.loc['mean', 'full_acc']
#mean_acc

# Looking at how much each solution dominates
Ideally need to find a measure which doesn't depend on ensemble size... variance?
I think mean across seeds, variance accross accuracy of members

In [12]:
mnvr = np.mean(np.var(acc_task_1, axis=1))
mnvr

0.00052245482197173

In [13]:
sys.path

['/Users/ferguscurrie/Documents/Uni/Research/Implementations',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/Users/ferguscurrie/Library/Python/3.9/lib/python/site-packages',
 '/usr/local/lib/python3.9/site-packages',
 '/usr/local/opt/python-tk@3.9/libexec',
 '/usr/local/lib/python3.9/site-packages/IPython/extensions',
 '/Users/ferguscurrie/.ipython']

# Preparing model vs dataset table

In [67]:
from sklearn.model_selection import train_test_split
def get_ytrain(j, t):
    dns = get_all_datasets()
    with open(f'results_file/{job}/{task}/{job}_{task}_info.txt') as f:
        data = f.read()
    for dn in dns:
        if dn in data:
            X, y = get_data(dn)
            seed = 169 * int(task)
            _, _, y_train, _ = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)
            return y_train


In [75]:
acc_all_tasks = []
max_nmembers = -1
for task in os.listdir(f'results_file/{job}'):
    # Not all done
    if len(os.listdir(f'results_file/{job}/{task}')) <= 1:
        continue 
    # Otherwise load it 
    scoring_task = load_all_scoring(f'results_file/{job}/{task}/scoring/') # (# seeds, # members in ensemble, # observatins)
    y = get_ytrain(job, task)
    acc_taskx = np.array(acc_task(scoring_task))
    acc_all_tasks.append(acc_taskx)
    if acc_taskx.shape[1] > max_nmembers:
        max_nmembers = acc_taskx.shape[1]